# Gridworld with First Visit

In [1]:
import numpy as np

# display output
from random import uniform
import random
import time
from IPython.display import display, clear_output

In [2]:
actions = [[-1, 0], [0, 1], [1, 0], [0, -1]] #up, right, down, left = (clockwise from up) 
action_count = len(actions) # total number of actions
gridSize = 5 # create a square grid of gridSize by gridSize
state_count = gridSize*gridSize # total number of states

In [3]:
class Gridworld():
    def __init__(self, gridSize):
        self.valueMap = np.zeros((gridSize, gridSize))
        self.states = [[i, j] for i in range(gridSize) for j in range(gridSize)]
        self.size = gridSize
        self.new_pos = [0, 0] # initialize new position for p_transition
        self.transition_prob = 1 # deterministic
    
    def initial_state(self):        # return initial state
        return grid.states[gridSize*gridSize-1]
   
    def transition_reward(self, current_pos, action): # return the transition probability

        # get next position: state: [0, 0], action: [0, 1], new_state = [0, 1]
        self.new_pos = np.array(current_pos) + np.array(action)

        # normally, reward = 0
        reward = 0

        # if new pos results in off the grid, return reward -1
        if -1 in self.new_pos or self.size in self.new_pos:
            reward = -1
        # if in state A, receive + 10
        if current_pos == [0, 1]:
            reward = 10
        # if in state B, receive + 5
        if current_pos == [0, 3]:
            reward = 5

        # if taking an action crosses the border; agent's new_pos is the same as the current pos
        if -1 in self.new_pos or self.size in self.new_pos: 
            self.new_pos = current_pos
            
        # if in state A, transition to state A'
        if current_pos == [0, 1]:
            self.new_pos = [4, 1]
            
        # if in state B, transition to state B'
        if current_pos == [0, 3]:
            self.new_pos = [2, 3]

        return self.new_pos, reward

## First-visit MC Control 

In [4]:
# create a grid object
grid = Gridworld(5)

In [5]:
# Initiate a random policy
random_policy = np.random.randint(1000, size=(state_count, action_count))
random_policy = random_policy/random_policy.sum(axis=1)[:,None]
policy = random_policy

In [6]:
# random policy
policy

array([[0.14808044, 0.43007313, 0.34963437, 0.07221207],
       [0.30545262, 0.25463208, 0.23451562, 0.20539968],
       [0.37001209, 0.20798065, 0.191052  , 0.23095526],
       [0.06037736, 0.32006861, 0.31423671, 0.30531732],
       [0.43959553, 0.07184673, 0.08568387, 0.40287387],
       [0.20419255, 0.34704969, 0.33385093, 0.11490683],
       [0.08866076, 0.40083994, 0.2165189 , 0.2939804 ],
       [0.27014973, 0.27556547, 0.15355209, 0.30073272],
       [0.15679562, 0.29887025, 0.20267032, 0.34166381],
       [0.19736842, 0.28767943, 0.28708134, 0.22787081],
       [0.04723779, 0.22417934, 0.00640512, 0.72217774],
       [0.17877448, 0.1269741 , 0.0846494 , 0.60960202],
       [0.20541205, 0.55350554, 0.11869619, 0.12238622],
       [0.24764543, 0.08144044, 0.35401662, 0.31689751],
       [0.42271563, 0.04739539, 0.11315115, 0.41673783],
       [0.6066105 , 0.05703176, 0.32663642, 0.00972132],
       [0.11976048, 0.2565638 , 0.45739291, 0.16628282],
       [0.30055579, 0.33219324,

### Create an Episode following Policy

In [7]:
def generate_episode(steps):

    # set initial state
    state_vector = grid.initial_state()

    # initialize state (with iniitial state), action list and reward list
    state_list = [state_vector]
    action_list = []
    reward_list = []

    # generate an episode
    for i in range(steps):

        # pick an action based on categorical distribution in policy
        action_index = int(np.random.choice(action_count, 1, p=policy[grid.states.index(state_vector)])) 
        action_vector = actions[action_index] # convert the integer index (ie. 0) to action (ie. [-1, 0])

        # get new state and reward after taking action from current state
        new_state_vector, reward = grid.transition_reward(state_vector, action_vector)
        state_vector = list(new_state_vector)

        # save state, action chosen and reward to list
        state_list.append(state_vector)
        action_list.append(action_vector)
        reward_list.append(reward)
        
    return state_list, action_list, reward_list

### First Visit MC

In [8]:
# initialize q values for all state action pairs
Q_values = np.zeros((state_count, action_count))

In [9]:
# intialize parameters
gamma = 0.99
epsilon = 0.2

In [10]:
# define average function
def Average(lst): 
    return sum(lst) / len(lst) 

In [11]:
# iterate 500 times: each time, generating an episode of 200 steps
max_steps = 200

# define variables for keeping track of time steps
Terminal = max_steps
t_list=[]
for i in range(1,max_steps+1):
    t = Terminal - i
    t_list.append(t)

In [12]:
# iteration 500 times
for iteration in range(500):
  
    # generate an episode of specified step count
    state_list, action_list, reward_list = generate_episode(max_steps)
    
#     print("state_list: ", state_list)
#     print("action_list: ", action_list)
#     print("reward_list: ", reward_list)
    
    # intialize G
    G = 0
    
    # initiate returns and visited list to none
    returns_list = []
    visited_list = []

    # loop for each step of episode: T-1, T-2, T-3 ... 0 = 199, 198, 197 ... 0
    for t in t_list:

        # calculate G: starting with the last reward at index t (naturally accounts for pseudocode's "t-1")
        G = gamma*G + reward_list[t]
        # print(G)
        
        # combine state action pair, for example, state = [0,0], action = [0,1], state_action_pair = [0,0,0,1]
        state_action_pair = []
        state_action_pair.extend(state_list[t])
        state_action_pair.extend(action_list[t])
        # print(state_action_pair)

        # check if state action pair have been visited before (if not: continue, else: move to the next time step)
        if state_action_pair not in visited_list:

            # add state action pair to visited list
            visited_list.append(state_action_pair)

            # append G to returns
            returns_list.append(G)

            # find state and action index, for example, converting action [-1, 0] to 0, and same for state #
            state_index = grid.states.index(state_list[t])
            action_index = actions.index(action_list[t])
    #         print("state_index: ", state_index)

            # write Q_values to the state-action pair
            Q_values[state_index][action_index] = Average(returns_list)
    #         print("average return: ", Average(returns_list))

            # choose best action at given state
            choose_action = np.argmax(Q_values[state_index])

            # overwrite policy
            for a in range(action_count): # for action in actions [0, 1, 2, 3]
                if choose_action == a: # if the choose_action is the same as the current action
                    policy[state_index][a] = 1 - epsilon + epsilon/action_count 
                else: # if choose_action is not the same as the current action
                    policy[state_index][a] = epsilon/action_count

In [13]:
# total unique state action pairs at the end of one episode
print(len(visited_list))

30


In [14]:
np.set_printoptions(precision=2)
Q_values

array([[  5.2 ,  -1.54,  -0.13,   4.78],
       [-11.36,   0.13,  -2.89, -14.3 ],
       [ -8.24,  -7.86,   0.  ,  -6.27],
       [ -0.85,   1.  ,  10.25,  -0.19],
       [ 13.24,  13.78,  16.45,  12.33],
       [ -0.19,   0.  ,   0.  ,  -0.07],
       [  0.54,   1.06,   0.  ,   0.  ],
       [ -8.85,   4.66, -32.13,   0.81],
       [  0.  ,   9.78,  13.76,   1.93],
       [ 13.34,  15.8 ,  14.34,   0.  ],
       [-13.89, -18.05,   0.  , -25.62],
       [  0.  , -13.13, -16.13, -28.12],
       [  1.26,   4.71,   0.  ,   0.  ],
       [  2.37,   0.  ,   3.35,   9.48],
       [  0.  ,  13.36,  17.24,  10.63],
       [ -0.24,   0.  ,   0.  ,  -0.36],
       [ -0.27,   0.  ,   0.  ,  -0.34],
       [  0.  ,   0.  ,   0.  ,   0.  ],
       [  2.78,  -1.94,   0.  ,   0.  ],
       [ -1.11,  -0.66,  -4.58,  16.86],
       [-10.48,  -0.54,  -6.1 , -21.56],
       [  0.  ,   1.44,  -0.11,  -0.58],
       [-36.29,  -0.17, -19.83,   0.  ],
       [  3.15,  -3.36,  -0.06,   0.  ],
       [ 17.74, 

In [15]:
#up, right, down, left = (clockwise from up) 
policy

array([[0.85, 0.05, 0.05, 0.05],
       [0.05, 0.85, 0.05, 0.05],
       [0.05, 0.05, 0.85, 0.05],
       [0.05, 0.05, 0.85, 0.05],
       [0.05, 0.05, 0.85, 0.05],
       [0.05, 0.85, 0.05, 0.05],
       [0.05, 0.85, 0.05, 0.05],
       [0.05, 0.85, 0.05, 0.05],
       [0.05, 0.05, 0.85, 0.05],
       [0.05, 0.85, 0.05, 0.05],
       [0.05, 0.05, 0.85, 0.05],
       [0.85, 0.05, 0.05, 0.05],
       [0.05, 0.85, 0.05, 0.05],
       [0.05, 0.05, 0.05, 0.85],
       [0.05, 0.05, 0.85, 0.05],
       [0.05, 0.85, 0.05, 0.05],
       [0.05, 0.85, 0.05, 0.05],
       [0.85, 0.05, 0.05, 0.05],
       [0.85, 0.05, 0.05, 0.05],
       [0.05, 0.05, 0.05, 0.85],
       [0.05, 0.85, 0.05, 0.05],
       [0.05, 0.85, 0.05, 0.05],
       [0.05, 0.05, 0.05, 0.85],
       [0.85, 0.05, 0.05, 0.05],
       [0.85, 0.05, 0.05, 0.05]])

## Visualize 

In [16]:
# PRINT POLICY TABLE ################################################################################
# import pandas library
import pandas as pd
# define column and index
columns=range(grid.size)
index = range(grid.size)
# define dataframe to represent policy table
policy_table = pd.DataFrame(index = index, columns=columns)

# iterate through policy to make a table that represents action number
# as action name (eg. left, right, up, down)
for state in range(len(policy)):
    
    # find the best action at each state
    best_action = np.argmax(policy[state])

    # get action name
    if best_action == 0:
        action_name = 'up'
    elif best_action == 1:
        action_name = 'right'
    elif best_action == 2:
        action_name = 'down'
    else:
        action_name = 'left'

    # calculate the row and column coordinate of the current state number
    row = int(state/grid.size)
    column = round((state/grid.size - int(state/grid.size))*grid.size)
            
    # assign action name
    policy_table.loc[row][column] = action_name

print("Policy Table: ")
print(policy_table)
print()

Policy Table: 
       0      1      2     3      4
0     up  right   down  down   down
1  right  right  right  down  right
2   down     up  right  left   down
3  right  right     up    up   left
4  right  right   left    up     up

